In [2]:
# Run this to download necessary libraries (If you have done this in the past in any notebook, you don't need to do this)
!pip install transformers[sentencepiece] # HG Transformer’s Library
!pip install datasets # HG Dataset’s Library
!pip install huggingface_hub # HG sharing Library
!pip install torch torchvision -U # Pytorch
# Remember to turn on T4 GPU Accelerator under 'Runtime'->'Change runtime type'
import torch

if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("Using CPU")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 21.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 96.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-

In [1]:
# Step 1 - Imports and Access Dataset
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from torch.utils.data import DataLoader

# Load the dataset
raw_datasets = load_dataset("glue", "mrpc")
print("raw_datasets", raw_datasets)
print(raw_datasets["train"].features)

ModuleNotFoundError: No module named 'datasets'

In [ ]:
# Step 2 Tokenize The Dataset

# Initialize the tokenizer and data collator
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
dataCollator = DataCollatorWithPadding(tokenizer=tokenizer)

# Define the tokenizing function
def tokenizeFunction(examples):
    # Adjusted to correctly pass pairs of sentences
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding=False, return_token_type_ids=True)

# Apply the tokenization function
tokenized_datasets = raw_datasets.map(tokenizeFunction, batched=True)

# This will remove unnecessary columns after tokenizing, however the Trainer API will do this for you
# tokenized_datasets = tokenized_datasets.remove_columns(["idx", "sentence1", "sentence2"])

# This will set the format to PyTorch tensors, however the Trainer API willl do this for you
# tokenized_datasets.set_format("torch")

print("tokenized_datasets: ",tokenized_datasets)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

tokenized_datasets:  DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})


In [ ]:
# Step 3 Use DataLoader and DataCollator to pad the tokens dyanamically
# For this example, we are only tokenizing the training data
# Set batch_size = *insert appropiate length*, shuffle = True , collate_fn = dataCollator
# DataLoader is a class (can't be renamed)
# dataCollator is an object of the DataCollator Class, and "dataCollator" is just a name
train_dataloader = DataLoader(tokenized_datasets["train"], batch_size=16, shuffle=True, collate_fn=dataCollator)

# Example of iterating
